In [3]:
import os
import glob
import re
import math
import copy
import pandas as pd
import heapq
from collections import defaultdict, deque

########################################################
# (A) subway_lines 및 그래프 생성
########################################################
subway_lines = {
    # (생략 – 기존 subway_lines 데이터)
    "1호선": [
        "연천","전곡","청산","소요산","동두천","보산","동두천중앙","지행","덕정","덕계","양주","녹양","가능","의정부","회룡","망월사","도봉산","도봉","방학","창동","녹천","월계","광운대","석계","신이문","외대앞","회기","청량리","제기동","신설동","동묘앞","동대문","종로5가","종로3가","종각","시청","서울역","남영","용산","노량진","대방","신길","영등포","신도림","구로",
        # 경인선 (구로 -> 인천)
        "구일","개봉","오류동","온수","역곡","소사","부천","중동","송내","부개","부평","백운","동암","간석","주안","도화","제물포","도원","동인천","인천",
        # 다시 구로 -> (경부선/장항선)
        "구로","가산디지털단지","독산","금천구청","석수","관악","안양","명학","금정","군포","당정","의왕","성균관대","화서","수원","세류","병점","세마","오산대","오산","진위","송탄","서정리","평택지제","평택","성환","직산","두정","천안","봉명","쌍용","아산","탕정","배방","온양온천","신창",
        "금천구청","광명",
        "병점","서동탄"
    ],
    "2호선": [
        "시청","을지로입구","을지로3가","을지로4가","동대문역사문화공원","신당","상왕십리","왕십리","한양대","뚝섬","성수","건대입구","구의","강변","잠실나루","잠실","잠실새내","종합운동장","삼성","선릉","역삼","강남","교대","서초","방배","사당","낙성대","서울대입구","봉천","신림","신대방","구로디지털단지","대림","신도림","문래","영등포구청","당산","합정","홍대입구","신촌","이대","아현","충정로","시청",
        "성수","용답","신답","용두","신설동",
        "신도림","도림천","양천구청","신정네거리","까치산"
    ],
    "3호선": [
        "대화","주엽","정발산","마두","백석","대곡","화정","원당","원흥","삼송","지축",
        "구파발","연신내","불광","녹번","홍제","무악재","독립문","경복궁","안국","종로3가","을지로3가","충무로","동대입구","약수","금호","옥수","압구정","신사","잠원","고속터미널","교대","남부터미널","양재","매봉","도곡","대치","학여울","대청","일원","수서","가락시장","경찰병원","오금"
    ],
    "4호선": [
        "진접","오남","별내별가람","당고개",
        "상계","노원","창동","쌍문","수유","미아","미아사거리","길음","성신여대입구","한성대입구","혜화","동대문","동대문역사문화공원","충무로","명동","회현","서울역","숙대입구","삼각지","신용산","이촌","동작","총신대입구(이수)","사당","남태령",
        "선바위","경마공원","대공원","과천","정부과천청사","인덕원","평촌","범계","금정",
        "산본","수리산","대야미","반월","상록수","한대앞","중앙","고잔","초지","안산","신길온천","정왕","오이도"
    ],
    "5호선": [
        "방화","개화산","김포공항","송정","마곡","발산","우장산","화곡","까치산","신정","목동","오목교","양평","영등포구청","영등포시장","신길","여의도","여의나루","마포","공덕","애오개","충정로","서대문","광화문","종로3가","을지로4가","동대문역사문화공원","청구","신금호","행당","왕십리","마장","답십리","장한평","군자","아차산","광나루","천호","강동","길동","굽은다리","명일","고덕","상일동","강일","미사","하남풍산","하남시청","하남검단산",
        "강동","둔촌동","올림픽공원","방이","오금","개롱","거여","마천"
    ],
    "6호선": [
        "응암","역촌","불광","독바위","연신내","구산","응암",
        "새절","증산","디지털미디어시티","월드컵경기장","마포구청","망원","합정",
        "상수","광흥창","대흥","공덕","효창공원앞","삼각지",
        "녹사평","이태원","한강진","버티고개","약수","청구","신당","동묘앞","창신","보문","안암","고려대","월곡","상월곡","돌곶이","석계","태릉입구","화랑대","봉화산","신내"
    ],
    "7호선": [
        "장암","도봉산","수락산","마들","노원","중계","하계","공릉","태릉입구","먹골","중화","상봉","면목","사가정","용마산","중곡","군자","어린이대공원","건대입구","자양","청담","강남구청","학동","논현","반포","고속터미널","내방","총신대입구(이수)","남성","숭실대입구","상도","장승배기","신대방삼거리","보라매","신풍","대림","남구로","가산디지털단지","철산","광명사거리","천왕","온수","까치울","부천종합운동장","춘의","신중동","부천시청","상동","삼산체육관","굴포천","부평구청","산곡","석남"
    ],
    "8호선": [
        "별내","다산","동구릉","구리","장자호수공원","암사역사공원",
        "암사","천호","강동구청","몽촌토성","잠실","석촌","송파","가락시장","문정","장지","복정","남위례",
        "산성","남한산성입구","단대오거리","신흥","수진","모란"
    ],
    "9호선": [
        "개화","김포공항","공항시장","신방화","마곡나루","양천향교","가양","증미","등촌","염창","신목동","선유도","당산",
        "국회의사당","여의도","샛강","노량진","노들","흑석","동작","구반포","신반포","고속터미널","사평","신논현",
        "언주","선정릉","삼성중앙","봉은사","종합운동장","삼전","석촌고분","석촌","송파나루","한성백제","올림픽공원","둔촌오륜","중앙보훈병원"
    ],
    "수인분당선": [
        "청량리","왕십리","서울숲","압구정로데오","강남구청","선정릉","선릉","한티","도곡","구룡","개포동","대모산입구","수서","복정","가천대","태평","모란","야탑","이매","서현","수내","정자","미금","오리","죽전","보정","구성","신갈","기흥","상갈","청명","영통","망포","매탄권선","수원시청","매교","수원",
        "고색","오목천","어천","야목","사리","한대앞","중앙","고잔","초지","안산","신길온천","정왕","오이도","달월","월곶","소래포구","인천논현","호구포","남동인더스파크","원인재","연수","송도","인하대","숭의","신포","인천"
    ],
    "신분당선": [
        "신사","논현","신논현","강남","양재","양재시민의숲","청계산입구","판교","정자","미금","동천","수지구청","성복","상현","광교중앙","광교"
    ],
    "경의중앙선": [
        "임진강","운천","문산","파주","월롱","금촌","금릉","운정","야당","탄현","일산","풍산","백마","곡산","대곡","능곡","행신","강매","한국항공대","수색","디지털미디어시티","가좌","신촌(경의중앙선)","서울역",
        "가좌","홍대입구","서강대","공덕","효창공원앞","용산","이촌","서빙고","한남","옥수","응봉","왕십리","청량리","회기","중랑","상봉","망우","양원","구리","도농","양정","덕소","도심","팔당","운길산","양수","신원","국수","아신","오빈","양평","원덕","용문","지평"
    ],
    "경춘선": [
        "청량리","회기","중랑","상봉","망우","신내","갈매","별내","퇴계원","사릉","금곡","평내호평","천마산","마석","대성리","청평","상천","가평","굴봉산","백양리","강촌","김유정","남춘천","춘천",
        "상봉","광운대"
    ],
    "서해선": [
        "일산","풍산","백마","곡산","대곡","능곡","김포공항","원종","부천종합운동장","소사","소새울","시흥대야","신천","신현","시흥시청","시흥능곡","달미","선부","초지","시우","원시"
    ],
    "공항철도": [
        "서울역","공덕","홍대입구","디지털미디어시티","마곡나루","김포공항","계양","검암","청라국제도시","영종","운서","공항화물청사","인천공항1터미널","인천공항2터미널"
    ],
    "우이신설선": [
        "북한산우이","솔밭공원","4.19민주묘지","가오리","화계","삼양","삼양사거리","솔샘","북한산보국문","정릉","성신여대입구","보문","신설동"
    ],
    "신림선": [
        "샛강","대방","서울지방병무청","보라매","보라매공원","보라매병원","당곡","신림","서원","서울대벤처타운","관악산"
    ],
    "김포골드라인": [
        "김포공항","고촌","풍무","사우","걸포북변","운양","장기","마산","구래","양촌"
    ],
    "인천1호선": [
        "계양","귤현","박촌","임학","계산","경인교대입구","작전","갈산","부평구청","부평시장","부평","동수","부평삼거리","간석오거리","인천시청","예술회관","인천터미널","문학경기장","선학","신연수","원인재","동춘","동막","캠퍼스타운","테크노파크","지식정보단지","인천대입구","센트럴파크","국제업무지구","송도달빛축제공원"
    ],
    "인천2호선": [
        "검단오류","왕길","검단사거리","마전","완정","독정","검암","검바위","아시아드경기장","서구청","가정","가정중앙시장","석남","서부여성회관","인천가좌","가재울","주안국가산단","주안","시민공원","석바위시장","인천시청","석천사거리","모래내시장","만수","남동구청","인천대공원","운연"
    ],
    "GTX-A": [
        "운정중앙","킨텍스","대곡","연신내","서울역",
        "수서","성남","구성","동탄"
    ],
    "의정부경전철": [
        "발곡","회룡","범골","경전철의정부","의정부시청","흥선","의정부중앙","동오","새말","경기도청북부청사","효자","곤제","어룡","송산","탑석"
    ],
    "용인경전철": [
        "기흥","강남대","지석","어정","동백","초당","삼가","시청/용인대","명지대","김량장","용인중앙시장(용인예술과학대)","고진","보평","둔전","전대/에버랜드"
    ],
    "경강선": [
        "판교","성남","이매","삼동","경기광주","초월","곤지암","신둔도예촌","이천","부발","세종대왕릉","여주"
    ],
}

# ─────────────────────────────────────────────
# [추가 함수 1] 방향 제어에 따른 간선 제거
# ─────────────────────────────────────────────
def remove_line_connection(graph, line_name, station_a, station_b, direction="both"):
    if direction == "both":
        if station_a in graph and station_b in graph[station_a]:
            if line_name in graph[station_a][station_b]:
                graph[station_a][station_b].remove(line_name)
                if not graph[station_a][station_b]:
                    del graph[station_a][station_b]
        if station_b in graph and station_a in graph[station_b]:
            if line_name in graph[station_b][station_a]:
                graph[station_b][station_a].remove(line_name)
                if not graph[station_b][station_a]:
                    del graph[station_b][station_a]
    elif direction == "forward":
        if station_a in graph and station_b in graph[station_a]:
            if line_name in graph[station_a][station_b]:
                graph[station_a][station_b].remove(line_name)
                if not graph[station_a][station_b]:
                    del graph[station_a][station_b]
    elif direction == "reverse":
        if station_b in graph and station_a in graph[station_b]:
            if line_name in graph[station_b][station_a]:
                graph[station_b][station_a].remove(line_name)
                if not graph[station_b][station_a]:
                    del graph[station_b][station_a]

# ─────────────────────────────────────────────
# [추가 함수 2] subway_lines 상의 구간을 찾아 해당 간선을 모두 제거
# ─────────────────────────────────────────────
def block_segment_in_graph(graph, line_name, start, end):
    """
    subway_lines에 정의된 해당 노선의 구간에서, start ~ end 사이의
    인접 간선들을 모두 제거(양방향)한다.
    """
    stations = subway_lines.get(line_name)
    if not stations:
        return
    try:
        idx_start = stations.index(start)
        idx_end = stations.index(end)
    except ValueError:
        return
    if idx_start <= idx_end:
        segment_stations = stations[idx_start:idx_end+1]
    else:
        segment_stations = stations[idx_end:idx_start+1][::-1]
    for i in range(len(segment_stations) - 1):
        sA = segment_stations[i]
        sB = segment_stations[i+1]
        remove_line_connection(graph, line_name, sA, sB, direction="both")

# ─────────────────────────────────────────────
# [추가 함수 3] 파일(날짜) 기준 개통 전 구간 차단
# ─────────────────────────────────────────────
def adjust_graph_for_openings(graph, file_date):
    """
    file_date: 정수형 YYYYMMDD
    아래 개통 정리(예시)를 참고하여, file_date보다 이전인 개통일의 구간은
    그래프에서 제거한다.
      - 20230701: 서해선 대곡 ~ 소사 개통  
      - 20230826: 서해선 일산 ~ 대곡 개통  
      - 20231216: 1호선 연천 ~ 소요산 개통  
      - 20240330: GTX-A 수서 ~ 성남, GTX-A 구성 ~ 동탄, 경강선 판교~성남 및 성남~이매 개통  
      - 20240629: GTX-A 성남 ~ 구성 개통  
      - 20240810: 8호선 별내 ~ 암사 개통  
      - 20241228: GTX-A 운정중앙 ~ 서울역 개통
    """
    conditions = [
        {"line": "서해선", "start": "대곡", "end": "소사", "open_date": 20230701},
        {"line": "서해선", "start": "일산", "end": "대곡", "open_date": 20230826},
        {"line": "1호선", "start": "연천", "end": "소요산", "open_date": 20231216},
        {"line": "GTX-A", "start": "수서", "end": "성남", "open_date": 20240330},
        {"line": "GTX-A", "start": "구성", "end": "동탄", "open_date": 20240330},
        {"line": "GTX-A", "start": "성남", "end": "구성", "open_date": 20240629},
        {"line": "경강선", "start": "판교", "end": "성남", "open_date": 20240330},
        {"line": "경강선", "start": "성남", "end": "이매", "open_date": 20240330},
        {"line": "8호선", "start": "별내", "end": "암사", "open_date": 20240810},
        {"line": "GTX-A", "start": "운정중앙", "end": "서울역", "open_date": 20241228},
    ]
    for cond in conditions:
        if file_date < cond["open_date"]:
            block_segment_in_graph(graph, cond["line"], cond["start"], cond["end"])
    return graph

# ─────────────────────────────────────────────
# [변경된 build_subway_graph]
# ─────────────────────────────────────────────
def build_subway_graph(subway_lines):
    graph = {}
    # (1) 모든 역 노드 생성
    for line_name, stations in subway_lines.items():
        for st in stations:
            if st not in graph:
                graph[st] = {}
    # (2) 인접 연결 (양방향)
    for line_name, stations in subway_lines.items():
        for i in range(len(stations)-1):
            A = stations[i]
            B = stations[i+1]
            if B not in graph[A]:
                graph[A][B] = set()
            graph[A][B].add(line_name)
            if A not in graph[B]:
                graph[B][A] = set()
            graph[B][A].add(line_name)
    # (3) 6호선 응암순환 단방향 처리 (역촌→응암 방향 제거)
    loop_6 = [
        ("응암","역촌"), ("역촌","불광"), ("불광","독바위"),
        ("독바위","연신내"), ("연신내","구산"), ("구산","응암")
    ]
    for (a, b) in loop_6:
        if b in graph and a in graph[b]:
            if "6호선" in graph[b][a]:
                graph[b][a].remove("6호선")
                if not graph[b][a]:
                    del graph[b][a]
    # (4) GTX-A 서울역↔수서 미개통 처리
    gtx_a_block_pairs = [("서울역", "수서")]
    for (stA, stB) in gtx_a_block_pairs:
        if stA in graph and stB in graph[stA]:
            if "GTX-A" in graph[stA][stB]:
                graph[stA][stB].remove("GTX-A")
                if not graph[stA][stB]:
                    del graph[stA][stB]
        if stB in graph and stA in graph[stB]:
            if "GTX-A" in graph[stB][stA]:
                graph[stB][stA].remove("GTX-A")
                if not graph[stB][stA]:
                    del graph[stB][stA]
    # (5) [추가] 특정 구간 제거 (제거 목록에 따라 방향 제어)
    removal_list = [
        ("6호선", "구산", "연신내", "forward"),
        ("6호선", "응암", "구산", "forward"),
        ("6호선", "역촌", "응암", "forward"),
        ("6호선", "불광", "역촌", "forward"),
        ("6호선", "독바위", "불광", "forward"),
        ("6호선", "연신내", "독바위", "forward"),
        ("1호선", "구로", "인천", "both"),
        ("1호선", "신창", "금천구청", "both"),
        ("1호선", "광명", "병점", "both"),
        ("2호선", "충정로", "성수", "both"),
        ("2호선", "신설동", "신도림", "both"),
        ("5호선", "하남검단산", "강동", "both"),
        ("1호선", "연천", "구일", "both"),
        ("1호선", "연천", "구로", "both"),
        ("1호선", "연천", "금천구청", "both"),
        ("1호선", "연천", "병점", "both"),
        ("1호선", "구일", "구로", "both"),
        ("1호선", "구일", "금천구청", "both"),
        ("1호선", "구일", "병점", "both"),
        ("1호선", "구로", "금천구청", "both"),
        ("1호선", "구로", "병점", "both"),
        ("1호선", "금천구청", "병점", "both"),
    ]
    for (line, sta_a, sta_b, direction) in removal_list:
        remove_line_connection(graph, line, sta_a, sta_b, direction)
    return graph

########################################################
# (B) 단일호선 BFS (환승 없음)
########################################################
def bfs_single_line(graph, line_name, start_station, end_station):
    if start_station not in graph or end_station not in graph:
        return []
    visited = set()
    queue = deque([[(start_station, line_name)]])
    while queue:
        path = queue.popleft()
        cur_st, cur_ln = path[-1]
        if cur_st == end_station:
            return path
        if cur_st not in visited:
            visited.add(cur_st)
            for nxt_st, lineset in graph[cur_st].items():
                if line_name in lineset:
                    newp = list(path)
                    newp.append((nxt_st, line_name))
                    queue.append(newp)
    return []

########################################################
# (C) (distance, transfers) 우선순위 큐 탐색
########################################################
def find_min_dist_min_transfer_path(graph, start_station, start_line, end_station, end_line):
    if start_station not in graph or end_station not in graph:
        return []
    visited = {}
    parent = {}
    start_state = (start_station, start_line)
    visited[start_state] = (0, 0)
    heap = []
    heapq.heappush(heap, (0, 0, start_state))
    while heap:
        cur_dist, cur_trans, (cur_st, cur_ln) = heapq.heappop(heap)
        if visited[(cur_st,cur_ln)] < (cur_dist, cur_trans):
            continue
        if cur_st == end_station and cur_ln == end_line:
            return reconstruct_route(parent, (cur_st, cur_ln))
        if cur_st not in graph:
            continue
        for nxt_st, lineset in graph[cur_st].items():
            if cur_ln in lineset:
                nxt_ln = cur_ln
                nxt_dist = cur_dist + 1
                nxt_trans = cur_trans
                nxt_state = (nxt_st, nxt_ln)
                old = visited.get(nxt_state, (999999, 999999))
                if (nxt_dist < old[0]) or (nxt_dist == old[0] and nxt_trans < old[1]):
                    visited[nxt_state] = (nxt_dist, nxt_trans)
                    parent[nxt_state] = (cur_st, cur_ln)
                    heapq.heappush(heap, (nxt_dist, nxt_trans, nxt_state))
            else:
                for possible_ln in lineset:
                    nxt_ln = possible_ln
                    nxt_dist = cur_dist + 1
                    nxt_trans = cur_trans + 1
                    nxt_state = (nxt_st, nxt_ln)
                    old = visited.get(nxt_state, (999999, 999999))
                    if (nxt_dist < old[0]) or (nxt_dist == old[0] and nxt_trans < old[1]):
                        visited[nxt_state] = (nxt_dist, nxt_trans)
                        parent[nxt_state] = (cur_st, cur_ln)
                        heapq.heappush(heap, (nxt_dist, nxt_trans, nxt_state))
    return []

def reconstruct_route(parent, end_state):
    route = []
    cur = end_state
    while cur in parent:
        route.append(cur)
        cur = parent[cur]
    route.append(cur)
    route.reverse()
    return route

# ─────────────────────────────────────────────
# (C-1) GTX-A 미사용 그래프 만들기
# ─────────────────────────────────────────────
def build_excluding_line_graph(graph, exclude_line):
    newg = {}
    for st, nbrs in graph.items():
        newg[st] = {}
        for nb, lineset in nbrs.items():
            filtered = {ln for ln in lineset if ln != exclude_line}
            if filtered:
                newg[st][nb] = filtered
    return newg

def find_min_dist_min_transfer_path_excluding_line(graph, exclude_line, start_st, start_ln, end_st, end_ln):
    newg = build_excluding_line_graph(graph, exclude_line)
    return find_min_dist_min_transfer_path(newg, start_st, start_ln, end_st, end_ln)

########################################################
# (D) 경로 압축: 연속된 환승 노선 합치기
########################################################
def compress_route(route):
    if not route:
        return []
    compressed = []
    cur_st, cur_lines = route[0][0], set([route[0][1]])
    for i in range(1, len(route)):
        st, ln = route[i]
        if st == cur_st:
            cur_lines.add(ln)
        else:
            compressed.append((cur_st, cur_lines))
            cur_st, cur_lines = st, set([ln])
    compressed.append((cur_st, cur_lines))
    return compressed

def format_compressed_route(compressed):
    parts = []
    for (st, lineset) in compressed:
        sorted_lines = sorted(lineset)
        lines_str = ",".join(sorted_lines)
        parts.append(f"{st}({lines_str})")
    return "->".join(parts)

########################################################
# (E) NEW DATA 매칭: 사용한 구간 추출
########################################################
def get_used_segments_in_path_for_newdata(graph, route):
    station_path = []
    prev_st = None
    for (st, ln) in route:
        if st != prev_st:
            station_path.append(st)
            prev_st = st
    used_segs = []
    for i in range(len(station_path)-1):
        A = station_path[i]
        B = station_path[i+1]
        if B in graph[A]:
            for ln in graph[A][B]:
                used_segs.append((A, B, ln))
    return used_segs

########################################################
# (F) df_main 처리: 경로 찾고 NEW DATA 매칭
########################################################
def process_row(graph, row):
    start_line = str(row["승차_호선"]).strip()
    start_st   = str(row["승차_역"]).strip()
    end_line   = str(row["하차_호선"]).strip()
    end_st     = str(row["하차_역"]).strip()
    # (A) 단일 호선 시도
    if start_line == end_line:
        single_route = bfs_single_line(graph, start_line, start_st, end_st)
        if single_route:
            return single_route
    # (B) 다중비용 탐색
    route = find_min_dist_min_transfer_path(graph, start_st, start_line, end_st, end_line)
    if not route:
        return []
    # (C) GTX-A 사용 여부 확인
    uses_gtx = any(ln == "GTX-A" for (st, ln) in route)
    if uses_gtx:
        route_no_gtx = find_min_dist_min_transfer_path_excluding_line(
            graph, "GTX-A", start_st, start_line, end_st, end_line
        )
        if route_no_gtx:
            if len(route) > 0.67 * len(route_no_gtx):
                route = route_no_gtx
    return route

def filter_df_main(df_main, graph, new_segments):
    results = []
    for idx, row in df_main.iterrows():
        route = process_row(graph, row)
        if not route:
            continue
        compressed = compress_route(route)
        final_path_str = format_compressed_route(compressed)
        used_segs = get_used_segments_in_path_for_newdata(graph, route)
        matched = [seg for seg in used_segs if seg in new_segments]
        if matched:
            row_dict = row.to_dict()
            row_dict["final_path"] = final_path_str
            row_dict["matched_newdata_segments"] = ";".join(f"{a}-{b}-{ln}" for (a,b,ln) in matched)
            results.append(row_dict)
    if not results:
        return pd.DataFrame(columns=list(df_main.columns)+["final_path","matched_newdata_segments"])
    else:
        return pd.DataFrame(results)

########################################################
# (G) 메인 처리: 날짜별 CSV/Excel 파일 쌍 처리 및 최종 필터링
########################################################
def main():
    # 1) 기본 지하철 그래프 생성 (개통 전/후 조정을 위해 base_graph로 사용)
    base_graph = build_subway_graph(subway_lines)
    
    # 2) 파일 경로 설정
    csv_dir = r"C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리 데이터셋"
    excel_dir = r"C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\0202최종\상위40퍼센트"

    # 3) CSV 파일 목록 및 날짜별 매핑 (파일명에서 마지막 8자리 추출)
    csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))
    csv_mapping = {}
    for path in csv_files:
        basename = os.path.basename(path)
        name_without_ext, _ = os.path.splitext(basename)
        date_str = name_without_ext[-8:]
        csv_mapping[date_str] = path

    # 4) Excel 파일 목록 및 날짜별 매핑
    excel_files = glob.glob(os.path.join(excel_dir, "*.xlsx"))
    excel_mapping = {}
    for path in excel_files:
        basename = os.path.basename(path)
        name_without_ext, _ = os.path.splitext(basename)
        date_str = name_without_ext[-8:]
        excel_mapping[date_str] = path

    # 5) 날짜 매칭: 두 폴더에서 공통 날짜만 사용
    common_dates = set(csv_mapping.keys()) & set(excel_mapping.keys())
    if not common_dates:
        print("매칭되는 날짜가 없습니다.")
        return

    # 6) 각 날짜별 처리
    for date in sorted(common_dates):
        csv_path = csv_mapping[date]
        excel_path = excel_mapping[date]
        print(f"처리 중 {date}: CSV: {csv_path}, Excel: {excel_path}")

        # CSV 파일 로딩 (df_main)
        df_main = pd.read_csv(csv_path, encoding="utf-8")

        # Excel 파일 로딩 및 'climate_pass'가 'X'인 데이터만 추출하여 new_segments 구성
        df_excel = pd.read_excel(excel_path)
        df_excel_filtered = df_excel[df_excel["climate_pass"] == "X"]
        new_segments = set()
        for i, row in df_excel_filtered.iterrows():
            A = str(row["start_station"]).strip()
            B = str(row["end_station"]).strip()
            L = str(row["lines"]).strip()
            new_segments.add((A, B, L))
        
        # 파일명 날짜(YYYYMMDD)를 정수형으로 변환하여 개통 시점 기준 그래프 조정
        try:
            file_date = int(date)
        except:
            file_date = 99999999  # 오류 발생 시 개통 조건 무시

        # base_graph에서 파일 날짜 기준 개통 전 구간 차단
        graph_adjusted = adjust_graph_for_openings(copy.deepcopy(base_graph), file_date)

        # df_main에 대해 필터 처리 (경로 탐색 및 NEW DATA 매칭)
        df_filtered = filter_df_main(df_main, graph_adjusted, new_segments)

        # ─────────────────────────────────────────────
        # [추가] 최종 데이터에서 "승차 가능"과 "하차 가능" 열에 대해
        # 조건을 만족하는 행 제거:
        # 1. 두 열 중 1개 이상 빈 값이 있을 때
        # 2. 두 열 모두 'O'인 경우
        # (열 이름은 CSV 파일의 열명에 맞춰 "승차 가능", "하차 가능" 사용)
        for col in ["승차 가능", "하차 가능"]:
            df_filtered[col] = df_filtered[col].fillna("").astype(str).str.strip()
        df_filtered = df_filtered[ ~ (
            (df_filtered["승차 가능"] == "") |
            (df_filtered["하차 가능"] == "") |
            ((df_filtered["승차 가능"] == "O") & (df_filtered["하차 가능"] == "O"))
        ) ]
        # ─────────────────────────────────────────────

        # 결과 저장 (날짜별 Excel 파일)
        output_filename = f"[0203최종]최종결과_각역노선포함_{date}.xlsx"
        df_filtered.to_excel(output_filename, index=False)
        print(f"[완료] {date}: 결과 {len(df_filtered)} 건, 저장: {output_filename}")

if __name__ == "__main__":
    main()


처리 중 20230331: CSV: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리 데이터셋\기동카 정리_지하철 역별 OD 20230331.csv, Excel: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\0202최종\상위40퍼센트\[0202최종]상위40퍼센트_구간정보_0번째_20230331.xlsx
[완료] 20230331: 결과 70001 건, 저장: [0203최종]최종결과_각역노선포함_20230331.xlsx
처리 중 20230430: CSV: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리 데이터셋\기동카 정리_지하철 역별 OD 20230430.csv, Excel: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\0202최종\상위40퍼센트\[0202최종]상위40퍼센트_구간정보_1번째_20230430.xlsx
[완료] 20230430: 결과 58243 건, 저장: [0203최종]최종결과_각역노선포함_20230430.xlsx
처리 중 20230531: CSV: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리 데이터셋\기동카 정리_지하철 역별 OD 20230531.csv, Excel: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\0202최종\상위40퍼센트\[0202최종]상위40퍼센트_구간정보_2번째_20230531.xlsx
[완료] 20230531: 결과 67853 건, 저장: [0203최종]최종결과_각역노선포함_20230531.xlsx
처리 중 20230630: CSV: C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리

KeyboardInterrupt: 